This file replicates Tables 1 and 2. We credit Jaroslav Borovička for the baseline code.

## Data processing

We will use data on asset returns and consumption growth. Several considerations are important.

* We need to agree on the length of the time period $t$. Typical period length for this type of problem would be a month, a quarter, or a year. Below, we consider quarterly frequency. This means that all quantities need to be downloaded or converted to quarterly frequency.

* Because the theory applies to real quantities and data on asset returns typically come in nominal form, we will need to deflate them by an appropriate inflation index.

* Since there are alternative time series for consumption growth, inflation, and many different types of available returns, appropriate choice of these quantities often involves considerable judgement.

You can download all data below manually into a single spreadsheet which you use for pre-processing, and then load the prepared spreadsheet directly for the GMM estimation. Below, I conduct all the steps in Python.


Import relevant packages.

In [1]:
# import packages
import numpy as np
import requests
import pandas as pd
import pandas_datareader as pdr
from io import BytesIO
from scipy import stats
from scipy import optimize

# load econutil package with some frequently used functions
import econutil as ec

Root package econutil imported.


### Consumption and inflation data from FRED

The Federal Reserve Bank of St. Louis maintains a large database of economic time series data (FRED - Federal Reserve Economic Data).

https://fred.stlouisfed.org/

Every time series is identified by a unique series identifier. Python provides API functionality in the <b><tt>pandas_datareader</tt></b> package to directly download the time series if you know the identifiers, or you can browse the database to find suitable data.

The theoretical moment conditions apply to real consumption. We could download nominal consumption expenditures and deflate them ourselves, or use one of the available real series. We do the latter. Here are some available series:

- <b><tt>PCEC</tt></b>: Personal consumption expenditures in billions of USD, seasonally adjusted (https://fred.stlouisfed.org/series/PCEC). This series is in nominal dollars but the theoretical moments are expressed in real consumption units, so we would need to deflate it by the proper price index.

- <b><tt>PCECC96</tt></b>: Personal consumption expenditures in billions of <b>chained 2017 USD</b>, seasonally adjusted (https://fred.stlouisfed.org/series/PCECC96). This series is already deflated to real terms. However, it expresses total consumption expenditures of the whole population, while the theory applies to an individual (even though representative) investor. We should therefore perhaps adjust for population growth, even though this will have negligible quantitative implications in our specific application.

- <b><tt>A794RX0Q048SBEA</tt></b>: Personal consumption expenditures <b>per capita</b> in <b>chained 2017 USD</b>, seasonally adjusted (https://fred.stlouisfed.org/series/A794RX0Q048SBEA).

- <b><tt>PCECTPI</tt></b>: Chain-type price index for personal consumption expenditures (https://fred.stlouisfed.org/series/PCECTPI). While we download consumption data already in real terms, we will need to deflate returns as well. Again, there are multiple price indices, the two most prominent ones being the Consumer Price Index (CPI), and the Personal Consumption Expenditures (PCE) Price Index, which are collected in alternative ways. We will use PCE to deflate returns, since the above consumption expenditures series are also deflated by PCE.

As you can see, there is a range of choices to be made when engaging in empirical work, some of which requiring judgement calls. For example, the differences between aggregate consumption and consumption per capita, or between using the CPI or PCE price indices will not have dramatic impacts for our conclusions but may be substantial in other applications.

In [2]:
# personal consumption PCEC, real personal consumption PCECC96,
# real personal consumption per capita A794RX0Q048SBEA, PCE price index PCECTPI

# myLoadDataFRED uses the pandas_reader.data.Reader function to download time series from FRED
data_C = ec.LoadDataFRED(series=['PCEC','PCECC96','A794RX0Q048SBEA','PCECTPI'],transform='none')

In [3]:
# create a YYYYQ (year, quarter) index for merging data with returns later
data_C = data_C['orig'].reset_index()
data_C['YYYYQ'] = data_C['DATE'].dt.year*10 + (data_C['DATE'].dt.month+2)//3
data_C = data_C.drop(columns=['DATE'])

# construct inflation time series ((P_{t}/P_{t-1}-1))
data_C['infl'] = data_C['PCECTPI'] + float("nan")
data_C.loc[1:,'infl'] =(data_C['PCECTPI'][1:].values / data_C['PCECTPI'][0:-1].values - 1)

# construct real consumption growth per capita ((C_t/C_{t-1}-1))
data_C['gC'] = data_C['A794RX0Q048SBEA'] + float("nan")
data_C.loc[1:,'gC'] =(data_C['A794RX0Q048SBEA'][1:].values / data_C['A794RX0Q048SBEA'][0:-1].values - 1)
data_C = data_C.drop(data_C.index[0])
#data_C = data_C.drop(data_C.index[data_C.index > 20200])

data_C = data_C.set_index('YYYYQ')

### Returns data from Kenneth French's website

Kenneth French maintains a large database of asset returns widely used as standard test returns in asset pricing applications. These emerged from his early work from Eugene Fama on factor based asset pricing.

https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/data_library.html

We will be using the most elementary dataset, containing the excess returns on three "Fama/French" factors:

https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/ftp/F-F_Research_Data_Factors_CSV.zip

These returns were originally used in Fama and French (1992), for a very brief introduction see the Wikipedia page on the <a href="https://en.wikipedia.org/wiki/Fama%E2%80%93French_three-factor_model">Fama-French three-factor model</a>.

The data set contains the following monthly returns, and their annual accumulated versions:

- <b><tt>RF</tt></b>: the nominal "risk-free" rate, equivalent to the yield on the U.S. one-month Treasury security

- <b><tt>Mkt-RF</tt></b>: the excess return on a broadly constructed U.S. stock portfolio, net of the risk-free rate

- <b><tt>SMB</tt></b>: the excess return on a portfolio of small U.S. firms, net of the return on a portfolio of large U.S. firms

- <b><tt>HML</tt></b>: the excess return on a portfolio of U.S. firms with high ratio of book valuation to market valuation, net of the return on a portfolio of U.S. firms with low ratio of book valuation to market valuation

We will be using the risk-free rate and the return on the aggregate stock market, the latter constructed from <b><tt>Mkt-RF</tt></b> by adding back the risk-free rate.

In our theory, the returns are real gross returns. The data are nominal monthly net returns, expressed in percent. Denote $t$ the quarter and $\widetilde{r}_{t+1,j}$, $j=1,2,3$ the monthly returns in the given quarter $t+1$, collected in the Kenneth French database.

We build the gross nominal quarterly return as

$$ R_{t+1}^N = \prod_{j=1}^3 \left( 1 + \frac{\widetilde{r}_{t+1,j}}{100} \right) $$

and then deflate by

$$ R_{t+1} = \frac{ R_{t+1}^N}{1+\pi_{t+1}}. $$

In [4]:
# download the data in ZIP format and extract the CSV file
param = {}
param['filename'] = 'https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/ftp/F-F_Research_Data_Factors_CSV.zip'
param['returns'] = ['Mkt-RF','SMB','HML','RF']

result = requests.get(param['filename'])
data_FF = pd.read_csv(BytesIO(result.content),compression='zip',header=2)

The file contains both monthly and annual data, so we first need to extract the right rows, corresponding to monthly data (this may be easier to do manually).

In [5]:
# process data
# rename date column
data_FF = data_FF.rename(columns={'Unnamed: 0':'date'})
# drop non-numerical rows
data_FF = data_FF[pd.to_numeric(data_FF['date'], errors='coerce').notnull()]
# only select data corresponding to months, and create a year-month index YYYYMM
data_FF['YYYYMM'] = pd.to_numeric(data_FF['date'])
data_FF = data_FF[data_FF['YYYYMM'] > 192606]
# convert returns columns into numerical values
for f in param['returns']:
    data_FF[f] = pd.to_numeric(data_FF[f])
# create years and months identifiers
data_FF['YYYY'] = data_FF['YYYYMM'] // 100
data_FF['MM'] = data_FF['YYYYMM'] % 100
# create the market return
data_FF['Mkt'] = data_FF['Mkt-RF'].to_numpy() + data_FF['RF']

The monthly return data must be converted to quarterly data by multiplicative accumulation of monthly returns in corresponding quarters.

In [6]:
# identify odd initial and terminal months not corresponding to whole quarters
init_year_months = sum(data_FF['YYYY'] == min(data_FF['YYYY']))
end_year_months = sum(data_FF['YYYY'] == max(data_FF['YYYY']))
# isolate months corresponding to whole quarters
if end_year_months % 3 > 0:
    d = data_FF.iloc[init_year_months % 3:-(end_year_months % 3),:].copy()
else:
    d = data_FF.iloc[init_year_months % 3:,:].copy()
# create a quarter index for each month
d['YYYYQ'] = d['YYYY']*10 + (d['MM']-1) // 3 + 1
# aggregate monthly nominal returns into quarterly nominal returns, using the groupby function
# returns are stored as net returns
# d = d.groupby('YYYYQ', group_keys=False).apply(lambda grp: ((1+grp[['RF','Mkt']]/100).prod()-1), include_groups=False)
d = d.groupby('YYYYQ', group_keys=False).apply(lambda grp: ((1+grp[['RF','Mkt']]/100).prod()-1))
#d = d.drop(columns=['YYYYQ'])

Finally, we can merge both datasets.

In [7]:
# merge the dataset from FRED with the returns data, only using overlapping quarters, using YYYYQ as the merging index
data = pd.concat([d,data_C],axis=1,join="inner")
data['date'] = data.index//10 + ((data.index % 10)-1)/4

# create real returns
data['Mkt_real'] = (1+data['Mkt'])/(1+data['infl'])-1
data['RF_real'] = (1+data['RF'])/(1+data['infl'])-1
data['Mkt-RF_real'] = data['Mkt_real'] - data['RF_real']

## Generalized method of moments (GMM)

In [8]:
data

,RF,Mkt,PCEC,PCECC96,A794RX0Q048SBEA,PCECTPI,infl,gC,date,Mkt_real,RF_real,Mkt-RF_real
YYYYQ,,,,,,,,,,,,
19472,0.000900,-0.006465,160.031,1373.880,9555,11.649,0.007961,0.012075,1947.25,-0.014312,-0.007005,-0.007308
19473,0.001200,0.018972,163.543,1378.358,9542,11.866,0.018628,-0.001361,1947.50,0.000338,-0.017109,0.017447
19474,0.002001,0.036693,167.672,1378.796,9501,12.162,0.024945,-0.004297,1947.75,0.011462,-0.022385,0.033847
19481,0.002302,-0.004967,170.372,1385.667,9510,12.297,0.011100,0.000947,1948.00,-0.015891,-0.008702,-0.007189
19482,0.002502,0.113741,174.142,1401.789,9582,12.424,0.010328,0.007571,1948.25,0.102357,-0.007746,0.110102
...,...,...,...,...,...,...,...,...,...,...,...,...
20234,0.013460,0.119431,19170.154,15781.367,46641,121.480,0.004108,0.006040,2023.75,0.114852,0.009314,0.105538
20241,0.013258,0.102034,19424.775,15856.867,46759,122.507,0.008454,0.002530,2024.00,0.092796,0.004764,0.088032
20242,0.013258,0.035713,19682.699,15967.266,46972,123.275,0.006269,0.004555,2024.25,0.029261,0.006946,0.022315


In [9]:
phi = 0.43     
mu  = 0.018     
delta = 0.036 

In [10]:
def GMM_objective_disaster_bet_gam_fixed(par):
    
    psi, d1, d2, p = par[0], par[1], par[2], par[3]

    ### ADDED CODE ###

    P = np.array([[phi, 1 - phi - d1, d1], 
                  [1 - phi - d2, phi, d2],
                  [0.5 - 0.5 * p, 0.5 - 0.5 * p, p]])

    GAMMAC = np.array([[1 + mu + delta, 1 + mu - delta, psi * (1 + mu)], 
                       [1 + mu + delta, 1 + mu - delta, psi * (1 + mu)],
                       [1 + mu + delta, 1 + mu - delta, psi * (1 + mu)]])
    
    GAMMAG = GAMMAC.copy()

    S = bet * np.power(GAMMAC, -gam)

    N = P.shape[0]
    I = np.identity(N)
    one = np.ones([N,1])
    
    # first compute the unconditional stationary distribution PII (assuming it is unique)
    # unconditional stationary distribution is the eigenvector of P' associated
    # with eigenvalue equal to 1 (the largest eigenvalue)
    eigval, eigvec = np.linalg.eig(P.transpose())
    idx = np.abs(eigval).argsort()
    eigval = eigval[idx]
    eigvec = eigvec[:,idx]
    PII = eigvec[:,-1:] / sum(eigvec[:,-1])

    # conditional gross risk-free rate (Nx1 vector)
    Rf = 1 / ((P * S) @ one)
    # unconditional gross risk-free rate, stored as a scalar
    ERf = (Rf.transpose() @ PII).item()
    
    # recursive formula for the price-dividend ratio: q =  P*S*GAMMAG @ (q + 1)
    # where * is elementwise multiplication, @ is matrix multiplication, 1 is an Nx1 vector of ones
    # solution given by
    # q = inv(I - P*S*GAMMAG) * (P*S*GAMMAG)*1

    # solution for the infinite-horizon asset only valid
    # if P*S*G has eigenvalues inside the unit circle
    M = P * S * GAMMAG
    eigval, eigvec = np.linalg.eig(M)
    maxeig = max(eigval)

    if (maxeig < 1):
        # asset price (Nx1 vector)
        q = np.linalg.inv(I - M) @ (M @ one)
        # returns (NxN matrix of returns R(i,j)
        R = ((1/q) @ (q.transpose()+1)) * GAMMAG
        # conditional expected returns (Nx1 vector)
        EtR = (R * P) @ one
        # unconditional expected return
        ER = (EtR.transpose() @ PII).item()
        # excess returns, realized and expected
        Re = R - np.tile(Rf,[1,N])
        EtRe = EtR - Rf
        ERe = ER - ERf
    else:
        q = (np.empty([N,1]))*np.nan
        R = (np.empty([N,N]))*np.nan
        EtR = (np.empty([N,1]))*np.nan
        ER = np.nan
        Re = (np.empty([N,N]))*np.nan
        EtRe = (np.empty([N,1]))*np.nan
        ERe = np.nan

    ### ORIGINAL CODE ###
    
    # moment conditions
    f1 = pd.Series([ERf - np.mean(1 + data['RF_real'])])
    
    
    VarRf = ((Rf.transpose() - ERf) ** 2 @ PII).item()
    f2 = pd.Series([VarRf - np.var(1 + data['RF_real'])])
    
    
    f3 = pd.Series([ER - np.mean(data['Mkt_real'])])
    

    EEtRe = (EtRe.transpose() @ PII).item()
    VarEtRe = ((EtRe.transpose() - EEtRe) ** 2 @ PII).item()

    EtRe2 = (Re ** 2 * P) @ one  
    VartRe = EtRe2 - EtRe ** 2
    EVartRe = (VartRe.transpose() @ PII).item()

    VarRe = EVartRe + VarEtRe
    
    f4 = pd.Series([VarRe - np.var(data['Mkt-RF_real'])])
    
    
    # number of data points
    T = len(f1)
    
    # weighting matrix - identity matrix chosen for simplicity
    W = np.identity(4) # SET FOR INITIAL RESULTS
    # W = np.array([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 0.01, 0], [0, 0, 0, 1]]) # SET FOR DOWNWEIGHTING f3
    
    # f is the T*M matrix of data for the moment conditions
    f = np.array([f1,f2,f3,f4]).transpose()
    
    # compute and return the GMM criterion function
    g = sum(f).transpose()/T
    
    return g.transpose()@W@g

In [11]:
# bet and gam to iterate through
bet_lst = np.arange(0, 1.1, 0.1).tolist()
gam_lst = np.arange(0, 11, 1).tolist()

In [12]:
import warnings
warnings.filterwarnings("ignore")

## Table 1

In [13]:
gmm_dict = {}
for bet in bet_lst:
    for gam in gam_lst:
        par = [0.85, 0.024, 0.024, 0]
        xopt = optimize.fmin(GMM_objective_disaster_bet_gam_fixed, par, xtol=1e-12, disp=False)
        psi, d1, d2, p = xopt[0], xopt[1], xopt[2], xopt[3]
        # Only print reasonable parameter values
        if (psi < 0.982 / 1.018) and (psi >= 0.65) and (d1 >= 0) and (d2 >= 0) and (p >= 0) and (d1 <= 0.081) and (d2 <= 0.081) and (p <= 1) and (d2 > d1):
            print(f"bet: {bet}, gam: {gam}, psi: {psi}, d1: {d1}, d2: {d2}, p: {p}")
            # Store result in dictionary
            gmm_dict[(bet, gam)] = (psi, d1, d2, p)

bet: 0.6000000000000001, gam: 0, psi: 0.7645760639590922, d1: 0.024996716673687077, d2: 0.025393521300485396, p: 0.0001652430858490285
bet: 0.6000000000000001, gam: 9, psi: 0.6963837658533261, d1: 0.05172341724566351, d2: 0.08057972904375449, p: 0.0315970573987443
bet: 0.7000000000000001, gam: 0, psi: 0.7010647102470347, d1: 0.024745172000960568, d2: 0.027066449260507976, p: 0.00024310693322758726
bet: 0.8, gam: 8, psi: 0.6554453163989522, d1: 0.012517441521696274, d2: 0.017507736860628703, p: 0.0350405854952667
bet: 0.9, gam: 10, psi: 0.7917543646695782, d1: 0.024584714553936765, d2: 0.05639964484604071, p: 2.7843196119782955e-05
bet: 1.0, gam: 1, psi: 0.8393749988825061, d1: 0.024149999979489516, d2: 0.025050000113056192, p: 3.124999821204522e-05
bet: 1.0, gam: 5, psi: 0.7100332069940407, d1: 0.019691637047806743, d2: 0.02622383886813628, p: 0.0001887839340000588
bet: 1.0, gam: 9, psi: 0.8263553705950404, d1: 0.013115345003117541, d2: 0.04856876862782025, p: 0.0001285072311641523
bet

In [14]:
def solve_Mehra_Prescott(model):
    
    S = model["SDF"]
    P = model["P"]
    GAMMAG = model["GAMMAG"]
    
    N = P.shape[0]
    I = np.identity(N)
    one = np.ones([N,1])
    
    # first compute the unconditional stationary distribution PII (assuming it is unique)
    # unconditional stationary distribution is the eigenvector of P' associated
    # with eigenvalue equal to 1 (the largest eigenvalue)
    eigval,eigvec = np.linalg.eig(P.transpose())
    idx = np.abs(eigval).argsort()
    eigval = eigval[idx]
    eigvec = eigvec[:,idx]
    PII = eigvec[:,-1:] /sum( eigvec[:,-1])

    # conditional gross risk-free rate (Nx1 vector)
    # Rf = 1 / (np.multiply(P,S)*one)
    Rf = 1 / ((P * S) @ one)
    # unconditional gross risk-free rate, stored as a scalar
    ERf = (Rf.transpose() @ PII).item()
    
    # recursive formula for the price-dividend ratio: q =  P*S*GAMMAG @ (q + 1)
    # where * is elementwise multiplication, @ is matrix multiplication, 1 is an Nx1 vector of ones
    # solution given by
    # q = inv(I - P*S*GAMMAG) * (P*S*GAMMAG)*1

    # solution for the infinite-horizon asset only valid
    # if P*S*G has eigenvalues inside the unit circle
    M = P*S*GAMMAG
    eigval,eigvec = np.linalg.eig(M)
    maxeig = max(eigval)

    if (maxeig < 1):
        # asset price (Nx1 vector)
        q = np.linalg.inv(I - M) @ (M @ one)
        # returns (NxN matrix of returns R(i,j)
        R = ((1/q) @ (q.transpose()+1)) * GAMMAG
        # conditional expected returns (Nx1 vector)
        EtR = (R * P) @ one
        # unconditional expected return
        ER = (EtR.transpose() @ PII).item()
        # excess returns, realized and expected
        Re = R - np.tile(Rf,[1,N])
        EtRe = EtR - Rf
        ERe = ER - ERf
    else:
        q = (np.empty([N,1]))*np.nan
        R = (np.empty([N,N]))*np.nan
        EtR = (np.empty([N,1]))*np.nan
        ER = np.nan
        Re = (np.empty([N,N]))*np.nan
        EtRe = (np.empty([N,1]))*np.nan
        ERe = np.nan

    sol = {"PII": PII, "Rf": Rf, "ERf": ERf, "q": q, "R": R, "EtR": EtR, "ER": ER, "Re": Re, "EtRe": EtRe, "ERe": ERe}
    return sol

## Table 2

In [15]:
# GMM ERf and ERe
for key in gmm_dict:

    model = {"PHI": 0.43, "MU": 0.018, "DELTA": 0.036}

    model["psi"], model["d1"], model["d2"], model["p"] = gmm_dict[key][0], gmm_dict[key][1], gmm_dict[key][2], gmm_dict[key][3]
    
    model["P"] = np.array([[model["PHI"], 1 - model["PHI"] - model["d1"], model["d1"]], 
                           [1 - model["PHI"] - model["d2"], model["PHI"], model["d2"]],
                           [0.5 - 0.5 * model["p"], 0.5 - 0.5 * model["p"], model["p"]]])
    
    model["GAMMAC"] = np.array([[1 + model["MU"] + model["DELTA"], 1 + model["MU"] - model["DELTA"], model["psi"] * (1 + model["MU"])], 
                                [1 + model["MU"] + model["DELTA"], 1 + model["MU"] - model["DELTA"], model["psi"] * (1 + model["MU"])],
                                [1 + model["MU"] + model["DELTA"], 1 + model["MU"] - model["DELTA"], model["psi"] * (1 + model["MU"])]])
    
    model["GAMMAG"] = model["GAMMAC"].copy()

    bet, gam = key[0], key[1]
    model["SDF"] = bet * np.power(model["GAMMAC"], -gam)
    sol = solve_Mehra_Prescott(model)

    ERf = sol["ERf"] - 1
    ERe = sol["ERe"]
    
    print(f"({bet},{gam}): {ERf}, {ERe}")

(0.6000000000000001,0): 0.6666666666666663, -4.440892098500626e-16
(0.6000000000000001,9): -0.24660169177174962, 0.258064647219718
(0.7000000000000001,0): 0.42857142857142816, -2.220446049250313e-16
(0.8,8): -0.023758467875154565, 0.03552120641951362
(0.9,10): -0.06751903967265405, 0.08024973004558744
(1.0,1): 0.012114906707942685, 0.0019448479525876472
(1.0,5): -0.024423127766000863, 0.03645347190662618
(1.0,9): -0.018298609996736492, 0.030882335919088733
(1.0,10): -0.010097289255448394, 0.022649153087565543


In [16]:
# Empirical ERf and ERe
for key in gmm_dict:

    model = {"PHI": 0.43, "MU": 0.018, "DELTA": 0.036}

    model["psi"], model["d1"], model["d2"], model["p"] = 0.85, 0.024, 0.024, 0
    
    model["P"] = np.array([[model["PHI"], 1 - model["PHI"] - model["d1"], model["d1"]], 
                           [1 - model["PHI"] - model["d2"], model["PHI"], model["d2"]],
                           [0.5 - 0.5 * model["p"], 0.5 - 0.5 * model["p"], model["p"]]])
    
    model["GAMMAC"] = np.array([[1 + model["MU"] + model["DELTA"], 1 + model["MU"] - model["DELTA"], model["psi"] * (1 + model["MU"])], 
                                [1 + model["MU"] + model["DELTA"], 1 + model["MU"] - model["DELTA"], model["psi"] * (1 + model["MU"])],
                                [1 + model["MU"] + model["DELTA"], 1 + model["MU"] - model["DELTA"], model["psi"] * (1 + model["MU"])]])
    
    model["GAMMAG"] = model["GAMMAC"].copy()

    bet, gam = key[0], key[1]
    model["SDF"] = bet * np.power(model["GAMMAC"], -gam)
    sol = solve_Mehra_Prescott(model)

    ERf = sol["ERf"] - 1
    ERe = sol["ERe"]
    
    print(f"({bet},{gam}): {ERf}, {ERe}")

(0.6000000000000001,0): 0.6666666666666665, 0.0
(0.6000000000000001,9): 0.7285727659775765, 0.04637161419880842
(0.7000000000000001,0): 0.4285714285714284, 2.220446049250313e-16
(0.8,8): 0.30360361152125814, 0.031336781616828624
(0.9,10): 0.14275694915873793, 0.040752672942820256
(1.0,1): 0.012590703662050284, nan
(1.0,5): 0.043828612246280985, 0.013350375328413211
(1.0,9): 0.037143659586545974, 0.032070461511460424
(1.0,10): 0.028481254242864074, 0.0380014219922975


## Examining moment condition errors

In [17]:
for key in gmm_dict:

    bet, gam = key[0], key[1]

    psi, d1, d2, p = gmm_dict[key][0], gmm_dict[key][1], gmm_dict[key][2], gmm_dict[key][3]

    P = np.array([[phi, 1 - phi - d1, d1], 
                  [1 - phi - d2, phi, d2],
                  [0.5 - 0.5 * p, 0.5 - 0.5 * p, p]])

    GAMMAC = np.array([[1 + mu + delta, 1 + mu - delta, psi * (1 + mu)], 
                       [1 + mu + delta, 1 + mu - delta, psi * (1 + mu)],
                       [1 + mu + delta, 1 + mu - delta, psi * (1 + mu)]])
    
    GAMMAG = GAMMAC.copy()

    S = bet * np.power(GAMMAC, -gam)

    N = P.shape[0]
    I = np.identity(N)
    one = np.ones([N,1])
    
    # first compute the unconditional stationary distribution PII (assuming it is unique)
    # unconditional stationary distribution is the eigenvector of P' associated
    # with eigenvalue equal to 1 (the largest eigenvalue)
    eigval, eigvec = np.linalg.eig(P.transpose())
    idx = np.abs(eigval).argsort()
    eigval = eigval[idx]
    eigvec = eigvec[:,idx]
    PII = eigvec[:,-1:] / sum(eigvec[:,-1])

    # conditional gross risk-free rate (Nx1 vector)
    Rf = 1 / ((P * S) @ one)
    # unconditional gross risk-free rate, stored as a scalar
    ERf = (Rf.transpose() @ PII).item()
    
    # recursive formula for the price-dividend ratio: q =  P*S*GAMMAG @ (q + 1)
    # where * is elementwise multiplication, @ is matrix multiplication, 1 is an Nx1 vector of ones
    # solution given by
    # q = inv(I - P*S*GAMMAG) * (P*S*GAMMAG)*1

    # solution for the infinite-horizon asset only valid
    # if P*S*G has eigenvalues inside the unit circle
    M = P * S * GAMMAG
    eigval, eigvec = np.linalg.eig(M)
    maxeig = max(eigval)

    if (maxeig < 1):
        # asset price (Nx1 vector)
        q = np.linalg.inv(I - M) @ (M @ one)
        # returns (NxN matrix of returns R(i,j)
        R = ((1/q) @ (q.transpose()+1)) * GAMMAG
        # conditional expected returns (Nx1 vector)
        EtR = (R * P) @ one
        # unconditional expected return
        ER = (EtR.transpose() @ PII).item()
        # excess returns, realized and expected
        Re = R - np.tile(Rf,[1,N])
        EtRe = EtR - Rf
        ERe = ER - ERf
    else:
        q = (np.empty([N,1]))*np.nan
        R = (np.empty([N,N]))*np.nan
        EtR = (np.empty([N,1]))*np.nan
        ER = np.nan
        Re = (np.empty([N,N]))*np.nan
        EtRe = (np.empty([N,1]))*np.nan
        ERe = np.nan
    
    # construct the moment conditions
    f1 = pd.Series([ERf - np.mean(1 + data['RF_real'])])
    
    
    VarRf = ((Rf.transpose() - ERf) ** 2 @ PII).item()
    f2 = pd.Series([VarRf - np.var(1 + data['RF_real'])])
    
    
    f3 = pd.Series([ER - np.mean(data['Mkt_real'])])
    

    EEtRe = (EtRe.transpose() @ PII).item()
    VarEtRe = ((EtRe.transpose() - EEtRe) ** 2 @ PII).item()

    EtRe2 = (Re ** 2 * P) @ one  
    VartRe = EtRe2 - EtRe ** 2
    EVartRe = (VartRe.transpose() @ PII).item()

    VarRe = EVartRe + VarEtRe
    
    f4 = pd.Series([VarRe - np.var(data['Mkt-RF_real'])])
    
    
    T = len(f1)
    f = np.array([f1,f2,f3,f4]).transpose()
    g = sum(f)/T
    
    print(f'({bet}, {gam}): Average moment condition errors (g, in %): {(g[0]*100):.5f}, {(g[1]*100):.5f}, {(g[2]*100):.5f},  {(g[3]*100):.5f}')

(0.6000000000000001, 0): Average moment condition errors (g, in %): 66.47194, -0.00468, 164.37021,  -0.00000
(0.6000000000000001, 9): Average moment condition errors (g, in %): -24.85490, 1.41568, 98.84984,  0.90338
(0.7000000000000001, 0): Average moment condition errors (g, in %): 42.66242, -0.00468, 140.56068,  0.00000
(0.8, 8): Average moment condition errors (g, in %): -2.57057, 0.16004, 98.87981,  -0.61548
(0.9, 10): Average moment condition errors (g, in %): -6.94663, 0.86122, 98.97661,  -0.11913
(1.0, 1): Average moment condition errors (g, in %): 1.01676, -0.00305, 99.10952,  -0.47288
(1.0, 5): Average moment condition errors (g, in %): -2.63704, 0.01948, 98.90657,  -0.22885
(1.0, 9): Average moment condition errors (g, in %): -2.02459, 0.17318, 98.96191,  -0.43361
(1.0, 10): Average moment condition errors (g, in %): -1.20446, 0.26345, 98.95873,  -0.49361
